In [ ]:
# 导入模块
import re
import base64

In [ ]:
# 主函数
def main():
    while True:
        print("DES加解密，请输入(1-加密 2-解密 0-退出):")
        DES_mode = input()
        if DES_mode == '1':
            print("请输入明文字符串，不能为空:")
            PlainText = input().replace(" ","")
            print("请输入密钥:")
            Key = input().replace(" ","")
            bin_CipherText = DES_encrypt(PlainText,Key)
#             CipherText = bin2str(bin_CipherText)
            print("加密后的二进制串: ",bin_CipherText)
        elif DES_mode == '2':
            print("请输入密文字符串，不能为空:")
            CipherText = input().replace(" ","")
            print("请输入密钥:")
            Key = input().replace(" ","")
            bin_PlainText = DES_decrypt(CipherText,Key)
#             PlainText = bin2str(bin_PlainText)
            print("解密后的二进制串: ",bin_PlainText)
        elif DES_mode == '0':
            print("exit...")
            break
        else:
            print("请重新输入！")
    return None

if __name__ == "__main__":
    main()

In [ ]:
def str2bin(message):
    '''字符串转二进制'''
    res = ""
    for i in message:
        tmp = bin(ord(i))[2:]
        for j in range(0,8-len(tmp)):
            tmp = '0'+ tmp   #把输出的b给去掉
        res += tmp
    return res
def bin2str(bin_str):
    '''二进制转字符串,不可用'''
    res = ""
#     用正则表达式分组
    tmp = re.findall(r'.{8}',bin_str)
    for i in tmp:
        res += chr(int(i,2))
    return res

In [ ]:
# 加解密部分
def ip_change(bin_str):
    '''IP盒处理'''
    res = ""
    for i in IP_table:
        res += bin_str[i-1]     #数组下标i-1
    return res

def ip_re_change(bin_str):
    '''IP逆盒处理'''
    res = ""
    for i in IP_re_table:
        res += bin_str[i-1]
    return res

def e_key(bin_str):
    '''E盒处理'''
    res = ""
    for i in E:
        res += bin_str[i-1]
    return res

def str_xor(my_str1,my_str2):
    '''字符串异或操作'''
    res = ""
    for i in range(0,len(my_str1)):
        #变成10进制是转化成字符串 2进制与10进制异或结果一样，都是1,0
        xor_res = int(my_str1[i],10)^int(my_str2[i],10) 
        if xor_res == 1:
            res += '1'
        if xor_res == 0:
            res += '0'
    return res

In [ ]:
def s_box(my_str):
    '''S盒过程'''
    res = ""
    c = 0
    for i in range(0,len(my_str),6):
        this_turn_str = my_str[i:i+6]
        row = int(this_turn_str[0]+this_turn_str[5],2)
        col = int(this_turn_str[1:5],2)
        #bin输出有可能不是4位str类型的值，通过下面的循环并且加上字符0
        num = bin(S[c][row*16 + col])[2:]   
        for gz in range(0,4-len(num)):
            num = '0'+ num
        res += num
        c  += 1
    return res

def p_box(bin_str):
    '''P盒置换'''
    res = ""
    for i in  P:
        res += bin_str[i-1]
    return res

def fun_f(bin_str,key):
    '''F函数'''
    first_output = e_key(bin_str)
    second_output = str_xor(first_output,key)
    third_output = s_box(second_output)
    last_output = p_box(third_output)
    return last_output

In [ ]:
# 密钥部分
def left_turn(my_str,num):
    '''循环左移操作'''
    move_part = my_str[num:len(my_str)]
    res = move_part + my_str[0:num]
    return res

def change_key1(my_key):
    '''秘钥的PC-1置换'''
    res = ""
    for i in PC_1:
        res += my_key[i-1]
    return res

def change_key2(my_key):
    '''秘钥的PC-2置换'''
    res  = ""
    for i in PC_2:
        res += my_key[i-1]
    return res

def gen_key(key):
    key_list = []
    divide_output = change_key1(key)
    key_C0 = divide_output[0:28]
    key_D0 = divide_output[28:]
    for i in SHIFT:
        key_c = left_turn(key_C0,i)
        key_d = left_turn(key_D0,i)
        key_output = change_key2(key_c + key_d)
        key_list.append(key_output)
    return key_list

In [ ]:
#简单判断以及处理信息分组
def deal_mess(bin_mess):
    """  
    :param bin_mess: 二进制的信息流
    :return: 补充的64位信息流
    """
    ans = len(bin_mess)
    if ans % 64 != 0:
        for i in range( 64 - (ans%64)):           #不够64位补充0
            bin_mess += '0'
    return bin_mess


#查看秘钥是否为64位
def input_key_judge(bin_key):
    """
    全部秘钥以补0的方式实现长度不满足64位的
    :param bin_key: 
    """
    ans = len(bin_key)
    if len(bin_key) < 64:
        if ans % 64 != 0:
            for i in range(64 - (ans % 64)):  # 不够64位补充0
                bin_key += '0'
    # else:
    #     bin_key = bin_key[0:64]    #秘钥超过64位的情况默认就是应该跟密文一样长 直接将密钥变为跟明文一样的长度，虽然安全性会有所下降
    return bin_key


In [ ]:
def des_encrypt_64bits(bin_message,bin_key): 
#     初始置换
    mes_ip_bin = ip_change(bin_message)
    print('初始IP置换结果:',mes_ip_bin)
#     生成16轮密钥
    key_lst = gen_key(bin_key)
#     分组
    mes_left = mes_ip_bin[0:32]
    mes_right = mes_ip_bin[32:]
    for i in range(0,15):
        print('第%d轮:'%(i+1))
        print('L%d:'%i,mes_left)
        print('R%d:'%i,mes_right)
        mes_tmp = mes_right
#         F函数和左分组异或
        f_result = fun_f(mes_tmp,key_lst[i])
        print('本轮密钥: ',key_lst[i])
        print('F函数结果: ',f_result)
        mes_right = str_xor(f_result,mes_left)
        mes_left = mes_tmp
        print('L%d:'%(i+1),mes_left)
        print('R%d:'%(i+1),mes_right)
    print('第16轮:')
    print('L15:',mes_left)
    print('R15:',mes_right)
    f_result = fun_f(mes_right,key_lst[15])
    print('F函数结果: ',f_result)
    mes_fin_left = str_xor(mes_left,f_result)
    mes_fin_right = mes_right
    print('L16:',mes_left)
    print('R16:',mes_right)
#     IP逆置换
    fin_message = ip_re_change(mes_fin_left + mes_fin_right)
    return fin_message


def des_decrypt_64bits(bin_mess,bin_key):
    mes_ip_bin = ip_change(bin_mess)
    #bin_key = input_key_judge(str2bin(key))
    key_lst = gen_key(bin_key)
    lst = range(1,16)
    cipher_left = mes_ip_bin[0:32]
    cipher_right = mes_ip_bin[32:]
    for i in lst[::-1]:
        mes_tmp = cipher_right
        cipher_right = str_xor(cipher_left,fun_f(cipher_right,key_lst[i]))
        cipher_left = mes_tmp
    fin_left = str_xor(cipher_left,fun_f(cipher_right,key_lst[0]))
    fin_right = cipher_right
    fin_output  = fin_left + fin_right
    bin_plain = ip_re_change(fin_output)
    res = bin2str(bin_plain)
    return res

In [ ]:
def DES_encrypt(message,key):
        bin_mess = deal_mess(str2bin(message))
#         bin_mess = deal_mess(message)
        res = ""
#         bin_key = input_key_judge(str2bin(key))
        bin_key = input_key_judge(key)
        tmp = re.findall(r'.{64}',bin_mess)
        group=1
        for i in tmp:
            print("第%d组二进制串:"%group)
            res += des_encrypt_64bits(i,bin_key)
            group+=1
        return res


def DES_decrypt(message,key):
    bin_mess = deal_mess(message)
    res = ""
    bin_key = input_key_judge(str2bin(key))
    tmp = re.findall(r'.{64}',bin_mess)
    for i in tmp:
        res += des_decrypt_64bits(i,bin_key)
    return res

In [ ]:
# DES Struct
IP_table = [58, 50, 42, 34, 26, 18, 10, 2,
            60, 52, 44, 36, 28, 20, 12, 4,
            62, 54, 46, 38, 30, 22, 14, 6,
            64, 56, 48, 40, 32, 24, 16, 8,
            57, 49, 41, 33, 25, 17, 9, 1,
            59, 51, 43, 35, 27, 19, 11, 3,
            61, 53, 45, 37, 29, 21, 13, 5,
            63, 55, 47, 39, 31, 23, 15, 7]


IP_re_table = [40,8, 48, 16, 56, 24, 64, 32, 39,
             7, 47, 15, 55, 23, 63, 31, 38, 6,
             46, 14, 54, 22, 62, 30, 37,5, 45,
             13, 53, 21, 61, 29, 36, 4, 44, 12,
             52, 20, 60, 28, 35, 3, 43, 11, 51,
             19, 59, 27, 34, 2, 42, 10, 50, 18,
             58, 26, 33, 1, 41,9, 49, 17, 57, 25]




E  = [32, 1,  2,  3,  4,  5,  4,  5,
       6, 7,  8,  9,  8,  9, 10, 11,
      12,13, 12, 13, 14, 15, 16, 17,
      16,17, 18, 19, 20, 21, 20, 21,
      22, 23, 24, 25,24, 25, 26, 27,
      28, 29,28, 29, 30, 31, 32,  1]



P = [16,  7, 20, 21, 29, 12, 28, 17,
     1, 15, 23, 26,  5, 18, 31, 10,
     2,  8, 24, 14, 32, 27,  3,  9,
     19, 13, 30, 6, 22, 11,  4,  25]


S =  [
 [14, 4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7,
     0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8,
     4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0,
     15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13 ],



[15,  1,  8, 14,  6, 11,  3,  4,  9,  7,  2, 13, 12,  0,  5, 10,
     3, 13,  4,  7, 15,  2,  8, 14, 12,  0,  1, 10,  6,  9, 11,  5,
     0, 14,  7, 11, 10,  4, 13,  1,  5,  8, 12,  6,  9,  3,  2, 15,
     13,  8, 10,  1,  3, 15,  4,  2, 11,  6,  7, 12,  0,  5, 14,  9],


[10,  0,  9, 14,  6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8,
     13,  7,  0,  9,  3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1,
     13,  6,  4,  9,  8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7,
     1, 10, 13,  0,  6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12 ],


[7, 13, 14,  3,  0,  6,  9, 10,  1,  2,  8,  5, 11,  12,  4, 15,
     13,  8, 11,  5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,9,
     10,  6,  9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4,
     3, 15,  0,  6, 10,  1, 13,  8,  9,  4,  5, 11, 12,  7,  2, 14],


 [2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9,
     14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3,  9,  8,  6,
     4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14,
     11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3],

[12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11,
     10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8,
     9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6,
     4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13],


[4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1,
     13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6,
     1,  4, 11, 13, 12,  3,  7, 14, 10, 15,  6,  8,  0,  5,  9,  2,
     6, 11, 13,  8,  1,  4, 10,  7,  9,  5,  0, 15, 14,  2,  3, 12],


[13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7,
     1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2,
     7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8,
     2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11],

]

#key

PC_1 = [57, 49, 41, 33, 25, 17,9,
       1, 58, 50, 42, 34, 26, 18,
      10,  2, 59, 51, 43, 35, 27,
      19, 11,  3, 60, 52, 44, 36,
      63, 55, 47, 39, 31, 23, 15,
       7, 62, 54, 46, 38, 30, 22,
      14,  6, 61, 53, 45, 37, 29,
      21, 13,  5, 28, 20, 12, 4]



PC_2 = [14, 17, 11, 24,  1,  5,  3, 28,
      15,  6, 21, 10, 23, 19, 12,  4,
      26,  8, 16,  7, 27, 20, 13,  2,
      41, 52, 31, 37, 47, 55, 30, 40,
      51, 45, 33, 48, 44, 49, 39, 56,
      34, 53, 46, 42, 50, 36, 29, 32]


#秘钥左移的位数
SHIFT = [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]

In [ ]:
01111001111010001001100111100100010110001000111000000100000110101000011100100101011110111110101111100001010111010000010001111001
1000011000010